In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [5]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [6]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [7]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [8]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

In [9]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [16]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 8s - loss: 3.2032 - val_loss: 3.1657 - 8s/epoch - 8s/step
Epoch 2/2000
1/1 - 0s - loss: 3.1614 - val_loss: 3.1427 - 148ms/epoch - 148ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1373 - val_loss: 3.1242 - 152ms/epoch - 152ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1174 - val_loss: 3.1082 - 154ms/epoch - 154ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.1000 - val_loss: 3.0936 - 142ms/epoch - 142ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.0841 - val_loss: 3.0807 - 143ms/epoch - 143ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.0698 - val_loss: 3.0697 - 142ms/epoch - 142ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.0573 - val_loss: 3.0597 - 156ms/epoch - 156ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.0455 - val_loss: 3.0505 - 145ms/epoch - 145ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.0346 - val_loss: 3.0416 - 161ms/epoch - 161ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.0242 - val_loss: 3.0318 - 146ms/epoch - 146ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.0133 - val_loss: 3.0208 - 138ms/epoch - 138ms/step
Epoch 1

Epoch 99/2000
1/1 - 0s - loss: 2.4294 - val_loss: 2.4383 - 167ms/epoch - 167ms/step
Epoch 100/2000
1/1 - 0s - loss: 2.4218 - val_loss: 2.4402 - 162ms/epoch - 162ms/step
Epoch 101/2000
1/1 - 0s - loss: 2.4236 - val_loss: 2.4312 - 195ms/epoch - 195ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.4148 - val_loss: 2.4286 - 164ms/epoch - 164ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.4126 - val_loss: 2.4255 - 178ms/epoch - 178ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.4095 - val_loss: 2.4192 - 162ms/epoch - 162ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.4031 - val_loss: 2.4169 - 158ms/epoch - 158ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.4015 - val_loss: 2.4116 - 156ms/epoch - 156ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.3964 - val_loss: 2.4092 - 149ms/epoch - 149ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.3935 - val_loss: 2.4051 - 148ms/epoch - 148ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.3897 - val_loss: 2.4016 - 141ms/epoch - 141ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.3869 - val_loss: 2.3982 - 141ms/

Epoch 196/2000
1/1 - 0s - loss: 2.2029 - val_loss: 2.2157 - 208ms/epoch - 208ms/step
Epoch 197/2000
1/1 - 0s - loss: 2.1983 - val_loss: 2.2156 - 238ms/epoch - 238ms/step
Epoch 198/2000
1/1 - 0s - loss: 2.1981 - val_loss: 2.2147 - 198ms/epoch - 198ms/step
Epoch 199/2000
1/1 - 0s - loss: 2.1963 - val_loss: 2.2110 - 201ms/epoch - 201ms/step
Epoch 200/2000
1/1 - 0s - loss: 2.1936 - val_loss: 2.2101 - 209ms/epoch - 209ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.1932 - val_loss: 2.2080 - 208ms/epoch - 208ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.1902 - val_loss: 2.2071 - 246ms/epoch - 246ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.1891 - val_loss: 2.2048 - 206ms/epoch - 206ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.1878 - val_loss: 2.2025 - 217ms/epoch - 217ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.1853 - val_loss: 2.2022 - 196ms/epoch - 196ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.1844 - val_loss: 2.1997 - 162ms/epoch - 162ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.1825 - val_loss: 2.1978 - 154ms

Epoch 293/2000
1/1 - 0s - loss: 2.0902 - val_loss: 2.1063 - 154ms/epoch - 154ms/step
Epoch 294/2000
1/1 - 0s - loss: 2.0895 - val_loss: 2.1057 - 157ms/epoch - 157ms/step
Epoch 295/2000
1/1 - 0s - loss: 2.0887 - val_loss: 2.1047 - 154ms/epoch - 154ms/step
Epoch 296/2000
1/1 - 0s - loss: 2.0880 - val_loss: 2.1041 - 157ms/epoch - 157ms/step
Epoch 297/2000
1/1 - 0s - loss: 2.0873 - val_loss: 2.1034 - 154ms/epoch - 154ms/step
Epoch 298/2000
1/1 - 0s - loss: 2.0866 - val_loss: 2.1027 - 159ms/epoch - 159ms/step
Epoch 299/2000
1/1 - 0s - loss: 2.0859 - val_loss: 2.1021 - 160ms/epoch - 160ms/step
Epoch 300/2000
1/1 - 0s - loss: 2.0852 - val_loss: 2.1014 - 165ms/epoch - 165ms/step
Epoch 301/2000
1/1 - 0s - loss: 2.0846 - val_loss: 2.1008 - 166ms/epoch - 166ms/step
Epoch 302/2000
1/1 - 0s - loss: 2.0839 - val_loss: 2.1001 - 167ms/epoch - 167ms/step
Epoch 303/2000
1/1 - 0s - loss: 2.0832 - val_loss: 2.0993 - 175ms/epoch - 175ms/step
Epoch 304/2000
1/1 - 0s - loss: 2.0826 - val_loss: 2.0988 - 166ms

Epoch 390/2000
1/1 - 0s - loss: 2.0435 - val_loss: 2.0611 - 160ms/epoch - 160ms/step
Epoch 391/2000
1/1 - 0s - loss: 2.0425 - val_loss: 2.0605 - 155ms/epoch - 155ms/step
Epoch 392/2000
1/1 - 0s - loss: 2.0423 - val_loss: 2.0601 - 156ms/epoch - 156ms/step
Epoch 393/2000
1/1 - 0s - loss: 2.0419 - val_loss: 2.0608 - 145ms/epoch - 145ms/step
Epoch 394/2000
1/1 - 0s - loss: 2.0415 - val_loss: 2.0606 - 144ms/epoch - 144ms/step
Epoch 395/2000
1/1 - 0s - loss: 2.0412 - val_loss: 2.0601 - 144ms/epoch - 144ms/step
Epoch 396/2000
1/1 - 0s - loss: 2.0409 - val_loss: 2.0599 - 157ms/epoch - 157ms/step
Epoch 397/2000
1/1 - 0s - loss: 2.0405 - val_loss: 2.0587 - 155ms/epoch - 155ms/step
Epoch 398/2000
1/1 - 0s - loss: 2.0402 - val_loss: 2.0586 - 157ms/epoch - 157ms/step
Epoch 399/2000
1/1 - 0s - loss: 2.0397 - val_loss: 2.0592 - 141ms/epoch - 141ms/step
Epoch 400/2000
1/1 - 0s - loss: 2.0398 - val_loss: 2.0586 - 145ms/epoch - 145ms/step
Epoch 401/2000
1/1 - 0s - loss: 2.0404 - val_loss: 2.0598 - 151ms

In [17]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [18]:
(np.mean(acc), np.mean(prob))

(1.0, 0.8643297)

In [13]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 2s - loss: 3.5863 - val_loss: 3.3188 - 2s/epoch - 2s/step
Epoch 2/2000
1/1 - 0s - loss: 3.3004 - val_loss: 3.2026 - 208ms/epoch - 208ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1773 - val_loss: 3.1451 - 199ms/epoch - 199ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1153 - val_loss: 3.1187 - 202ms/epoch - 202ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.0840 - val_loss: 3.1055 - 221ms/epoch - 221ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.0647 - val_loss: 3.1001 - 227ms/epoch - 227ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.0529 - val_loss: 3.0925 - 227ms/epoch - 227ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.0395 - val_loss: 3.0780 - 227ms/epoch - 227ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.0203 - val_loss: 3.0621 - 211ms/epoch - 211ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.0009 - val_loss: 3.0505 - 231ms/epoch - 231ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.9868 - val_loss: 3.0449 - 223ms/epoch - 223ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.9793 - val_loss: 3.0441 - 211ms/epoch - 211ms/step
Epoch 1

Epoch 99/2000
1/1 - 0s - loss: 2.1640 - val_loss: 2.2683 - 168ms/epoch - 168ms/step
Epoch 100/2000
1/1 - 0s - loss: 2.1487 - val_loss: 2.2545 - 162ms/epoch - 162ms/step
Epoch 101/2000
1/1 - 0s - loss: 2.1344 - val_loss: 2.2416 - 155ms/epoch - 155ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.1199 - val_loss: 2.2287 - 157ms/epoch - 157ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.1056 - val_loss: 2.2146 - 166ms/epoch - 166ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.0917 - val_loss: 2.2002 - 165ms/epoch - 165ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.0774 - val_loss: 2.1863 - 167ms/epoch - 167ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.0633 - val_loss: 2.1727 - 160ms/epoch - 160ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.0500 - val_loss: 2.1599 - 173ms/epoch - 173ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.0377 - val_loss: 2.1492 - 176ms/epoch - 176ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.0271 - val_loss: 2.1402 - 173ms/epoch - 173ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.0185 - val_loss: 2.1323 - 168ms/

In [14]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [15]:
(np.mean(acc), np.mean(prob))

(1.0, 0.9771991)